In [14]:
import pandas as pd
import os
import random

In [15]:
df = pd.read_csv('primaryschool.csv')

In [16]:
df.describe()

,Time,Person 1,Person 2
count,125773.000000,125773.000000,125773.000000
mean,90687.675574,1635.858109,1737.303897
std,43793.491033,122.818086,137.779067
min,31220.000000,1426.000000,1427.000000
25%,45640.000000,1545.000000,1653.000000
50%,118580.000000,1661.000000,1759.000000
75%,132020.000000,1719.000000,1857.000000
max,148120.000000,1920.000000,1922.000000


In [17]:
timestamps = (df["Time"].iloc[-1] - df["Time"].iloc[0])/20
print(timestamps)

5845.0


195 timestamps are going to be clustered together and considered as one day.<br>
This would make the dataset into 30 days

In [28]:
class SIR:
    suscepted = []
    infected = []
    recovered = []
    # Can be modified according to the disease scenario
    alpha = 0.8
    beta = 0.3
    initial_infected = 12
    
#     As per the data in the dataset
    start_range = 1426
    end_range = 1922+1
    
    def random_sample(self,start_range,end_range,select):
        return random.sample(range(start_range,end_range),select)
    
    def init(self):
        for person in range(self.start_range,self.end_range):
            self.suscepted.append(person)
        self.infected = self.random_sample(self.start_range,self.end_range,self.initial_infected)
        
        for infected_person in self.infected:
            self.suscepted.remove(infected_person)
    def person_type(self, person):
        if(person in self.suscepted):
            return 'suscepted'
        if(person in self.infected):
            return 'infected'
        return 'recovered'
    
    def get_new_infected(self, infected_contact):
        total = len(infected_contact)
        new_infected_total = self.alpha*total
        new_infected = random.sample(infected_contact,k = int(new_infected_total))
        
        # Remove duplicates
        new_infected = list(set(new_infected))
        
        # Add them to infected
        for infected_person in new_infected:
            self.infected.append(infected_person)
            self.suscepted.remove(infected_person)
    
    def get_new_recovered(self):
        recovered_count = self.beta*(len(self.infected))
        new_recovered = random.sample(self.infected,k = int(recovered_count))
        
        # Add them to recovered
        for recovered_person in new_recovered:
            self.infected.remove(recovered_person)
            self.recovered.append(recovered_person)
        
        
        
        

In [29]:
model = SIR()
model.init()

In [30]:
total_count = 0
while total_count < df.shape[0]:
    count = 0
    infected_contact = []
    while count < 195 and total_count < df.shape[0]:
        person1 = df["Person 1"][total_count]
        person2 = df["Person 2"][total_count]
        
        # Check for transitions from Suscepted to Infected
        if model.person_type(person1) == 'suscepted' and model.person_type(person2) == 'infected':
            infected_contact.append(person1)
        if model.person_type(person2) == 'suscepted' and model.person_type(person1) == 'infected':
            infected_contact.append(person2)
        
        count = count + 1
        total_count = total_count + 1
    model.get_new_recovered()
    model.get_new_infected(infected_contact)
    print("After Day ",total_count/195)
    print("Number of susceptible: ",len(model.suscepted))
    print("Number of infected: ",len(model.infected))
    print("Number of recovered: ",len(model.recovered))
    
    
            
        
        

After Day  1.0
Number of susceptible:  481
Number of infected:  13
Number of recovered:  3
After Day  2.0
Number of susceptible:  479
Number of infected:  12
Number of recovered:  6
After Day  3.0
Number of susceptible:  479
Number of infected:  9
Number of recovered:  9
After Day  4.0
Number of susceptible:  477
Number of infected:  9
Number of recovered:  11
After Day  5.0
Number of susceptible:  470
Number of infected:  14
Number of recovered:  13
After Day  6.0
Number of susceptible:  468
Number of infected:  12
Number of recovered:  17
After Day  7.0
Number of susceptible:  468
Number of infected:  9
Number of recovered:  20
After Day  8.0
Number of susceptible:  468
Number of infected:  7
Number of recovered:  22
After Day  9.0
Number of susceptible:  466
Number of infected:  7
Number of recovered:  24
After Day  10.0
Number of susceptible:  466
Number of infected:  5
Number of recovered:  26
After Day  11.0
Number of susceptible:  464
Number of infected:  6
Number of recovered: 

After Day  96.0
Number of susceptible:  344
Number of infected:  3
Number of recovered:  150
After Day  97.0
Number of susceptible:  344
Number of infected:  3
Number of recovered:  150
After Day  98.0
Number of susceptible:  344
Number of infected:  3
Number of recovered:  150
After Day  99.0
Number of susceptible:  344
Number of infected:  3
Number of recovered:  150
After Day  100.0
Number of susceptible:  344
Number of infected:  3
Number of recovered:  150
After Day  101.0
Number of susceptible:  344
Number of infected:  3
Number of recovered:  150
After Day  102.0
Number of susceptible:  344
Number of infected:  3
Number of recovered:  150
After Day  103.0
Number of susceptible:  344
Number of infected:  3
Number of recovered:  150
After Day  104.0
Number of susceptible:  344
Number of infected:  3
Number of recovered:  150
After Day  105.0
Number of susceptible:  344
Number of infected:  3
Number of recovered:  150
After Day  106.0
Number of susceptible:  344
Number of infected:

After Day  186.0
Number of susceptible:  305
Number of infected:  3
Number of recovered:  189
After Day  187.0
Number of susceptible:  305
Number of infected:  3
Number of recovered:  189
After Day  188.0
Number of susceptible:  305
Number of infected:  3
Number of recovered:  189
After Day  189.0
Number of susceptible:  305
Number of infected:  3
Number of recovered:  189
After Day  190.0
Number of susceptible:  305
Number of infected:  3
Number of recovered:  189
After Day  191.0
Number of susceptible:  305
Number of infected:  3
Number of recovered:  189
After Day  192.0
Number of susceptible:  305
Number of infected:  3
Number of recovered:  189
After Day  193.0
Number of susceptible:  305
Number of infected:  3
Number of recovered:  189
After Day  194.0
Number of susceptible:  305
Number of infected:  3
Number of recovered:  189
After Day  195.0
Number of susceptible:  305
Number of infected:  3
Number of recovered:  189
After Day  196.0
Number of susceptible:  305
Number of infec

After Day  282.0
Number of susceptible:  294
Number of infected:  4
Number of recovered:  199
After Day  283.0
Number of susceptible:  294
Number of infected:  3
Number of recovered:  200
After Day  284.0
Number of susceptible:  294
Number of infected:  3
Number of recovered:  200
After Day  285.0
Number of susceptible:  294
Number of infected:  3
Number of recovered:  200
After Day  286.0
Number of susceptible:  294
Number of infected:  3
Number of recovered:  200
After Day  287.0
Number of susceptible:  294
Number of infected:  3
Number of recovered:  200
After Day  288.0
Number of susceptible:  294
Number of infected:  3
Number of recovered:  200
After Day  289.0
Number of susceptible:  294
Number of infected:  3
Number of recovered:  200
After Day  290.0
Number of susceptible:  294
Number of infected:  3
Number of recovered:  200
After Day  291.0
Number of susceptible:  294
Number of infected:  3
Number of recovered:  200
After Day  292.0
Number of susceptible:  294
Number of infec

After Day  372.0
Number of susceptible:  287
Number of infected:  5
Number of recovered:  205
After Day  373.0
Number of susceptible:  287
Number of infected:  4
Number of recovered:  206
After Day  374.0
Number of susceptible:  287
Number of infected:  3
Number of recovered:  207
After Day  375.0
Number of susceptible:  287
Number of infected:  3
Number of recovered:  207
After Day  376.0
Number of susceptible:  287
Number of infected:  3
Number of recovered:  207
After Day  377.0
Number of susceptible:  287
Number of infected:  3
Number of recovered:  207
After Day  378.0
Number of susceptible:  287
Number of infected:  3
Number of recovered:  207
After Day  379.0
Number of susceptible:  287
Number of infected:  3
Number of recovered:  207
After Day  380.0
Number of susceptible:  287
Number of infected:  3
Number of recovered:  207
After Day  381.0
Number of susceptible:  287
Number of infected:  3
Number of recovered:  207
After Day  382.0
Number of susceptible:  287
Number of infec

After Day  470.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  471.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  472.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  473.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  474.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  475.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  476.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  477.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  478.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  479.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  480.0
Number of susceptible:  283
Number of infec

After Day  564.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  565.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  566.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  567.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  568.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  569.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  570.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  571.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  572.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  573.0
Number of susceptible:  283
Number of infected:  3
Number of recovered:  211
After Day  574.0
Number of susceptible:  283
Number of infec